# HELP - Lockman SWIRE Master List Creation

This notebook presents the creation of the HELP master list on the Lockman SWIRE field.

<table>
  <tr>
    <th>Survey/Telescope/Instrument</th>
    <th>Filters</th>
    <th>Location</th>
    <th>Notes</th>
  </tr>
  <tr>
    <td>UKIDSS DXS/UKIRT/WFCAM/</td>
    <td>J,K</td>
    <td>dmu0_UKIDSS-DXS, dmu0_ELAIS-N1_DXS</td>
    <td></td>
  </tr>
  <tr>
    <td>SWIRE/Spitzer/IRAC MIPS</td>
    <td>IRAC234 & MIPS123</td>
    <td>dmu0_DataFusion-Spitzer</td>
    <td></td>
  </tr>
  <tr>
    <td>SERVS/Spitzer/IRAC		</td>
    <td></td>
    <td>dmu0_DataFusion-Spitzer</td>
    <td></td>
  </tr>
  <tr>
    <td>PS1 3PS/Pan-STARRS1</td>
    <td>grizy</td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>PS1 MDS/Pan-STARRS1</td>
    <td>grizy</td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>SpARCS/CFHT/MegaCam</td>
    <td></td>
    <td>dmu0_SpARCS</td>
    <td></td>
  </tr>
  <tr>
    <td>INT/WFC				</td>
    <td>u,g,r,i,z</td>
    <td>dmu0_INTWFC</td>
    <td></td>
  </tr>
</table>

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))